In [6]:
# =============================================================================
# STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION
# =============================================================================

!pip install ultralytics opencv-python numpy

print("\n🎥 STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION")

import cv2
import json
import numpy as np
from ultralytics import YOLO
from datetime import datetime

class ParkingVisualizer:
    def __init__(self, model_path, region_json, video_path, output_path):
        """
        YOLOv8-based Parking Visualization System
        ------------------------------------------
        model_path: Path to YOLOv8 model (e.g., yolov8n.pt or custom-trained model)
        region_json: JSON file defining parking zones (annotated points)
        video_path: Path to input video (drone footage or CCTV)
        output_path: Path where annotated output video will be saved
        """
        self.model = YOLO(model_path)
        self.video_path = video_path
        self.output_path = output_path
        
        # Load parking regions
        with open(region_json, 'r') as f:
            self.regions = json.load(f)

        self.colors = {
            'occupied': (0, 0, 255),  # Red for occupied
            'available': (0, 255, 0)  # Green for available
        }

    def is_inside_region(self, point, polygon):
        """Check if a detected object's center point lies inside a polygon region."""
        return cv2.pointPolygonTest(np.array(polygon, np.int32), point, False) >= 0

    def run(self):
        """Run visualization process."""
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print("❌ Error: Unable to open video file.")
            return
        
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Video writer
        out = cv2.VideoWriter(
            self.output_path,
            cv2.VideoWriter_fourcc(*'mp4v'),
            fps,
            (frame_width, frame_height)
        )

        print("🚗 Processing video frames for parking visualization...")
        frame_count = 0
        total_spots = len(self.regions)
        start_time = datetime.now()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            detections = self.model(frame, verbose=False)
            boxes = detections[0].boxes.xyxy.cpu().numpy() if detections and detections[0].boxes else []

            # Track occupancy
            region_status = {str(i): 'available' for i in range(len(self.regions))}
            for box in boxes:
                x1, y1, x2, y2 = map(int, box[:4])
                cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
                for i, region in enumerate(self.regions):
                    if self.is_inside_region((cx, cy), region['points']):
                        region_status[str(i)] = 'occupied'

            # Count available spots
            occupied = list(region_status.values()).count('occupied')
            available = total_spots - occupied
            occupancy_rate = (occupied / total_spots) * 100

            # Draw parking regions
            for i, region in enumerate(self.regions):
                pts = np.array(region['points'], np.int32)
                status = region_status[str(i)]
                color = self.colors[status]
                cv2.polylines(frame, [pts], True, color, 2)
                cv2.putText(
                    frame, f"{status.upper()}",
                    (pts[0][0], pts[0][1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2
                )

            # Display occupancy metrics
            cv2.rectangle(frame, (10, 10), (350, 70), (50, 50, 50), -1)
            cv2.putText(frame, f"Total Spots: {total_spots}", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"Available: {available}  Occupied: {occupied}", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame, f"Occupancy: {occupancy_rate:.1f}%", (200, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            out.write(frame)
            frame_count += 1

            # Display live (optional)
            cv2.imshow("Smart Parking Visualization", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        duration = (datetime.now() - start_time).seconds
        print(f"✅ Visualization complete — saved to {self.output_path}")
        print(f"🖼️ Processed {frame_count} frames in {duration}s")
        print(f"📊 Final Occupancy Rate: {occupancy_rate:.1f}% ({occupied}/{total_spots} occupied)")

# =============================================================================
# Example Usage
# =============================================================================

model_path = "models/yolov8n.pt"          # Your YOLOv8 model (replace if custom)
region_json = "file/parking_zones.json"   # Annotated regions (4-point polygons)
video_path = "file/sample_video.mp4"      # Parking lot or drone video
output_path = "output/parking_visual.mp4" # Annotated video output

visualizer = ParkingVisualizer(model_path, region_json, video_path, output_path)
visualizer.run()


StatementMeta(, ac1fa5d8-b91e-4017-b0fa-f58a05e0ce1c, 8, Finished, Cancelled, Cancelled)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 954.5 kB/s eta 0:00:00 0:00:01
INFO: pip is looking at multiple versions of matplotlib to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.1 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 18.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 21.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ModuleNotFoundError: No module named 'cv2'